In [82]:
# packages
import numpy as np
import pandas as pd
import os
import sys
sys.path.append(os.path.dirname(os.getcwd()) + '/source')
import pickle
from src.PDE_Solver import PDESolver_nd
from src.supportfunctions import finiteDiff, PDESolver_2d
import matplotlib.pyplot as plt
import matplotlib as mpl
from multiprocessing import Pool
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False
mpl.rcParams["figure.edgecolor"] = "w"
mpl.rcParams['figure.facecolor'] = "w"
mpl.rcParams["savefig.dpi"] = 300

In [83]:
# parameters
delta        = 0.01
eta          = 0.032
xi_a         = 0.01
theta_list   = pd.read_csv("../ClimateUncertaintySpillover_ov/data/model144.csv", header=None)[0].to_numpy()
theta_list   = theta_list/1000
# damage function
y_bar        = 2
gamma_1      = 0.00017675
gamma_2      = 2*0.0022
gamma_3_list = np.linspace(0., 1./3, 3)
# y grid
# λ = np.log(2)/6
lambda0      = 10

In [84]:
y1_step = .02
y1_grid = np.arange(0., 4., y1_step)

y2_step = .001
y2_grid = np.arange(0., .05, y2_step)

(y1_mat, y2_mat)= np.meshgrid(y1_grid, y2_grid, indexing = 'ij')
stateSpace      = np.hstack([y1_mat.reshape(-1,1, order='F'), y2_mat.reshape(-1,1,order='F')])

hy1  = y1_grid[1] - y1_grid[0]
hy2  = y2_grid[1] - y2_grid[0]
n_y1 = len(y1_grid)
n_y2 = len(y2_grid)
pi_ao = np.ones((len(theta_list), n_y1, n_y2))/len(theta_list)
pi_a = pi_ao
theta_mat = np.zeros((len(theta_list), n_y1, n_y2))

for i in range(len(theta_list)):
        theta_mat[i] = theta_list[i]
Df = gamma_1 + gamma_1*y1_mat + gamma_3_list[5]*(y1_mat - y_bar)*(y1_mat >y_bar)
E_new = eta*np.ones(y1_mat.shape)
E_old = E_new
tol=1e-7
epsilon=1
max_iter=2000
fraction=0.05

In [85]:
def solve_prep(y1_mat, y2_mat, gamma_3, tol=1e-7, ϵ=1, max_iter=2000, fraction=0.05): 
    log_prob_ratio = np.zeros_like(theta_mat)
    w              = np.zeros_like(theta_mat)
    V_new =  - eta*((y1_mat+y2_mat) + (y1_mat+y2_mat)**2)
    lhs_error = 0.5
    count = 0
    while lhs_error > tol and count < max_iter:
        V=V_new
        E=E_new
        #Apply Finite Difference 
        dVdy1 = finiteDiff(V, 0,1,hy1)
        dVdy2 = finiteDiff(V, 1,1,hy2)
        #computation of controls using backwards and forward difference
        for i in range(len(theta_mat)):
            log_prob_ratio[i] =- dVdy2 *E* theta_mat[i,:,:] /xi_a +1
            w[i]              = pi_a[i]*np.exp(log_prob_ratio[i])
            #Use the uncertainty subjective probabilites to compute 
        E = dVdy2 * sum(theta_mat*w)
        E[E <= 1e-10] = 1e-10
        #Computation of the Coeffecients 
        A = np.ones_like(y1_mat)*(-delta)
        B_y1 = y2_mat
        B_y2 = -y2_mat + sum(theta_mat*w)*E
        C_yy1 = np.zeros(y1_mat.shape)
        C_yy2 = np.zeros(y1_mat.shape)
        D = (eta-1)/delta * (gamma_1+gamma_2*y1_mat + gamma_3*(y1_mat-y_bar)\
                      * (y1_mat>y_bar))*y2_mat+np.log(E)*eta \
                      + xi_a*sum(w*np.log(w)-np.log(pi_a))
        out = PDESolver_2d(stateSpace, A, B_y1, B_y2, C_yy1, C_yy2, D, V, epsilon, solverType = 'False Transient')
        out_comp = out[2].reshape(V.shape,order = "F")
        rhs = A*V + B_y1*dVdy1 + B_y2*dVdy2  + D
        rhs_error = np.max(abs(rhs))
        lhs_error = np.max(abs((out_comp - V)/epsilon))
        count = count+ 1
        V_new = out_comp
        E_old = E_new
    print("Episode {:d}: PDE Error: {:.12f}; False Transient Error: {:.12f}; Iterations: {:d}; CG Error: {:.12f}".format(count, rhs_error, lhs_error, out[0], out[1]))
    result = dict(V=V, E=E, w=w)
    return result 

In [86]:
def ode_y_parallel(args_list):
    with Pool() as p:
        res_list = p.starmap(solve_prep, args_list)
    return res_list        

def postjump_list(args_list):
    res_list = ode_y_parallel(args_list)
    ϕ_list = np.zeros((len(gamma_3_list), len(y1_grid), len(y2_grid)))
    ems_list = np.zeros((len(gamma_3_list), len(y1_grid), len(y2_grid)))
    for j in range(len(gamma_3_list)):
        ϕ_list[j] = res_list[j]['v0']
        ems_list[j] = res_list[j]['ems']
        
    return ϕ_list, ems_list

args_list_test  = []
for gamma_3_i in gamma_3_list:
    args_iter = (y1_mat, y2_mat, gamma_3_i, 1e-6, 0.001, 2000, 0.05)
    args_list_test.append(args_iter)
ϕ_list, ems_list = postjump_list(args_list_test)

KeyboardInterrupt: 

In [78]:
from scipy import interpolate
def simulation(y1_grid, y2_grid, e_grid, λ, θ=1.86/1000., y1_0=.5394, y2_0=1.86/1000, T=100):
    e_fun = interpolate.interp2d(y1_grid, y2_grid, e_grid.T)
    Et = np.zeros(T+1)
    y1t = np.zeros(T+1)
    y2t = np.zeros(T+1)
    for i in range(T+1):
#         y2_0 = max(y2_0, 0)
#         y2_0 = min(y2_0, 0.05)
        Et[i] = e_fun(y1_0, y2_0)
        y1t[i] = y1_0
        y2t[i] = y2_0
        y2_0 = np.exp(-λ)*y2_0 + (1 - np.exp(-λ))*θ*Et[i] 
#         y2_0 = max(y2_0, 0)
        y1_0 = y1_0 + y2_0
    return Et, y1t, y2t

In [87]:
et2, y1t2, y2t2 = simulation(y1_grid, y2_grid, ems_list[0], lambda0, theta=np.mean(theta), y1_0=2)

NameError: name 'ems_list' is not defined